<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#EXERCICE 1
# Installation des dépendances requises
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.21

# Création du dossier cache (ignorer l'erreur si existe déjà)
!mkdir -p cache

# Imports nécessaires
import pandas as pd
import numpy as np
from IPython.display import display
import requests
import io
import zipfile

print("=== Chargement et préparation des données ===")

# Téléchargement et extraction du dataset
url = "https://github.com/devtlv/Datasets-GEN-AI-Bootcamp/raw/refs/heads/main/Week%207/W7D4/labelled_newscatcher_dataset.zip"
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Lecture du fichier avec des paramètres explicites
with zip_file.open("labelled_newscatcher_dataset.csv") as csv_file:
    # Lecture avec pandas en spécifiant les paramètres de parsing
    pdf = pd.read_csv(
        csv_file,
        sep=',',           # Séparateur
        quotechar='"',     # Caractère de citation
        escapechar='\\',   # Caractère d'échappement
        on_bad_lines='skip' # Ignorer les lignes problématiques
    )

# Ajout de l'ID comme demandé
pdf["id"] = pdf.index.astype(str)

# Affichage du DataFrame comme demandé
print("\n=== Aperçu du DataFrame complet ===")
print(f"Dimensions du DataFrame: {pdf.shape}")
display(pdf.head())

# Inspection de la structure
print("\n=== Structure du DataFrame ===")
print("\nInformations sur les colonnes:")
pdf.info()

print("\nStatistiques descriptives:")
display(pdf.describe())

print("\nTypes de données:")
print(pdf.dtypes)

# Création du subset de 1000 lignes
subset_size = 1000
pdf_subset = pdf.head(subset_size)

print(f"\n=== Sous-ensemble ({subset_size} premières lignes) ===")
print(f"Taille du sous-ensemble: {len(pdf_subset)} lignes")
display(pdf_subset.head())

# Vérification des valeurs manquantes
print("\n=== Analyse des valeurs manquantes ===")
missing_values = pdf.isnull().sum()
print("Valeurs manquantes par colonne:")
display(missing_values[missing_values > 0] if missing_values.sum() > 0 else "Aucune valeur manquante")


=== Chargement et préparation des données ===

=== Aperçu du DataFrame complet ===
Dimensions du DataFrame: (82817, 2)


,topic;link;domain;published_date;title;lang,id
0,SCIENCE;https://www.eurekalert.org/pub_release...,0
1,SCIENCE;https://www.express.co.uk/news/science...,1
2,SCIENCE;https://www.ndtv.com/world-news/glacie...,2
3,SCIENCE;https://www.thesun.ie/tech/5742187/per...,3
4,SCIENCE;https://interestingengineering.com/nas...,4



=== Structure du DataFrame ===

Informations sur les colonnes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82817 entries, 0 to 82816
Data columns (total 2 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   topic;link;domain;published_date;title;lang  82817 non-null  object
 1   id                                           82817 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB

Statistiques descriptives:


,topic;link;domain;published_date;title;lang,id
count,82817,82817
unique,82817,82817
top,SCIENCE;https://www.eurekalert.org/pub_release...,0
freq,1,1



Types de données:
topic;link;domain;published_date;title;lang    object
id                                             object
dtype: object

=== Sous-ensemble (1000 premières lignes) ===
Taille du sous-ensemble: 1000 lignes


,topic;link;domain;published_date;title;lang,id
0,SCIENCE;https://www.eurekalert.org/pub_release...,0
1,SCIENCE;https://www.express.co.uk/news/science...,1
2,SCIENCE;https://www.ndtv.com/world-news/glacie...,2
3,SCIENCE;https://www.thesun.ie/tech/5742187/per...,3
4,SCIENCE;https://interestingengineering.com/nas...,4



=== Analyse des valeurs manquantes ===
Valeurs manquantes par colonne:


'Aucune valeur manquante'

In [ ]:
# Import des bibliothèques nécessaires
from sentence_transformers import InputExample, SentenceTransformer
import pandas as pd
import numpy as np

print("=== Exercise 2 : Vectorisation avec Sentence Transformers ===")

# Vérifions d'abord la structure des données
pdf_subset = pdf.head(1000)  # Utilisation des 1000 premières lignes

# Affichage d'une ligne pour comprendre la structure
print("Structure d'une ligne de données :")
print(pdf_subset.iloc[0])

# Nettoyage des données avec vérification
# Séparons d'abord une ligne pour voir la structure
sample_split = pdf_subset['topic;link;domain;published_date;title;lang'].iloc[0].split(';')
print("\nNombre de colonnes après séparation :", len(sample_split))
print("Contenu après séparation :", sample_split)

# Maintenant faisons la séparation correcte
columns_data = pdf_subset['topic;link;domain;published_date;title;lang'].str.split(';', expand=True)
print("\nNombre de colonnes obtenues :", len(columns_data.columns))

# Attribution des noms de colonnes en fonction du nombre réel de colonnes
actual_columns = ['topic', 'link', 'domain', 'published_date', 'title', 'lang'][:len(columns_data.columns)]
columns_data.columns = actual_columns

# Définition de la fonction helper
def example_create_fn(doc1: pd.Series) -> InputExample:
    """
    Helper function that outputs a sentence_transformer guid, label, and text
    """
    return InputExample(
        guid=str(hash(doc1)),  # Identifiant unique basé sur le titre
        texts=[doc1],          # Le titre de l'article
        label=None            # Pas de label nécessaire pour cet exercice
    )

# Création des exemples d'entraînement avec la colonne title
print("\n=== Création des exemples d'entraînement ===")
faiss_train_examples = columns_data['title'].apply(example_create_fn).tolist()

# Affichage des 10 premiers exemples
print("\nAperçu des 10 premiers exemples :")
for example in faiss_train_examples[:10]:
    print(f"GUID: {example.guid}")
    print(f"Texte: {example.texts[0]}\n")

# Initialisation du modèle
print("\n=== Initialisation du modèle ===")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Extraction des titres et génération des embeddings
titles = columns_data['title'].tolist()
print("\n=== Génération des embeddings ===")
faiss_title_embedding = model.encode(titles, show_progress_bar=True)

# Affichage des dimensions
print("\n=== Dimensions des embeddings ===")
print(len(faiss_title_embedding), len(faiss_title_embedding[0]))


=== Exercise 2 : Vectorisation avec Sentence Transformers ===
Structure d'une ligne de données :
topic;link;domain;published_date;title;lang    SCIENCE;https://www.eurekalert.org/pub_release...
id                                                                                             0
Name: 0, dtype: object

Nombre de colonnes après séparation : 6
Contenu après séparation : ['SCIENCE', 'https://www.eurekalert.org/pub_releases/2020-08/dbnl-acl080620.php', 'eurekalert.org', '2020-08-06 13:59:45', "A closer look at water-splitting's solar fuel potential", 'en']

Nombre de colonnes obtenues : 7


ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [ ]:
# EXERCICE 2

# Import des bibliothèques nécessaires
from sentence_transformers import InputExample, SentenceTransformer
import pandas as pd
import numpy as np

print("=== Exercise 2 : Vectorisation avec Sentence Transformers ===")

# Préparation des données
pdf_subset = pdf.head(1000)  # Utilisation des 1000 premières lignes

# Séparation des colonnes avec analyse du résultat
columns_data = pdf_subset['topic;link;domain;published_date;title;lang'].str.split(';', expand=True)

# Vérification du nombre de colonnes et ajustement des noms
print(f"\nNombre de colonnes après séparation : {len(columns_data.columns)}")
actual_columns = ['topic', 'link', 'domain', 'published_date', 'title', 'lang'] + [f'extra_{i}' for i in range(len(columns_data.columns)-6)]
columns_data.columns = actual_columns

# Définition de la fonction helper comme demandé dans l'exercice
def example_create_fn(doc1: pd.Series) -> InputExample:
    """
    Helper function that outputs a sentence_transformer guid, label, and text
    """
    return InputExample(
        guid=str(hash(doc1)),  # Identifiant unique basé sur le titre
        texts=[doc1],          # Le titre de l'article
        label=None            # Pas de label nécessaire pour cet exercice
    )

# Création des exemples d'entraînement
print("\n=== Création des exemples d'entraînement ===")
faiss_train_examples = columns_data['title'].apply(example_create_fn).tolist()

# Affichage des 10 premiers exemples comme demandé
print("\nLes 10 premiers exemples :")
faiss_train_examples[:10]

# Initialisation du modèle all-MiniLM-L6-v2
print("\n=== Initialisation du modèle ===")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Extraction des titres et conversion en liste
titles = columns_data['title'].tolist()

# Génération des embeddings
print("\n=== Génération des embeddings ===")
faiss_title_embedding = model.encode(titles, show_progress_bar=True)

# Affichage des dimensions comme demandé dans l'exercice
print("\n=== Dimensions des embeddings ===")
print(len(faiss_title_embedding), len(faiss_title_embedding[0]))



=== Exercise 2 : Vectorisation avec Sentence Transformers ===

Nombre de colonnes après séparation : 7

=== Création des exemples d'entraînement ===

Les 10 premiers exemples :

=== Initialisation du modèle ===


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== Génération des embeddings ===


Batches:   0%|          | 0/32 [00:00<?, ?it/s]


=== Dimensions des embeddings ===
1000 384


In [ ]:
# Import des bibliothèques nécessaires
import numpy as np
import faiss
from IPython.display import display

print("=== Exercise 3 : FAISS Indexing and Search ===")

# Préparation des données pour l'indexation
pdf_to_index = columns_data  # Utilisation du DataFrame de l'exercice 2
id_index = np.array(pdf_to_index.index.astype(int))  # Conversion des index en array numpy

# Normalisation des embeddings pour la recherche par similarité cosinus
content_encoded_normalized = faiss_title_embedding.copy()  # Copie des embeddings
faiss.normalize_L2(content_encoded_normalized)  # Normalisation L2

# Création de l'index FAISS
print("\n=== Création de l'index FAISS ===")
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))
index_content.add_with_ids(content_encoded_normalized, id_index)

print(f"Nombre de vecteurs dans l'index : {index_content.ntotal}")

# Définition de la fonction de recherche
def search_content(query, pdf_to_index, k=3):
    """
    Recherche les documents les plus similaires à la requête
    Args:
        query: texte de la requête
        pdf_to_index: DataFrame contenant les documents
        k: nombre de résultats à retourner
    Returns:
        DataFrame avec les résultats et scores de similarité
    """
    # Encodage de la requête
    query_vector = model.encode([query])
    query_vector = query_vector.reshape(1, -1)
    faiss.normalize_L2(query_vector)

    # Recherche des k plus proches voisins
    similarities, indices = index_content.search(query_vector, k)

    # Création du DataFrame de résultats
    results = pdf_to_index.iloc[indices[0]]
    results = results.copy()  # Pour éviter les warnings de SettingWithCopyWarning
    results["similarities"] = similarities[0]

    return results

# Test de la fonction avec la requête "animal"
print("\n=== Test de recherche ===")
print("Recherche pour la requête 'animal':")
results = search_content("animal", pdf_to_index, k=5)
display(results)

# Affichage des informations sur les résultats
print("\n=== Statistiques de recherche ===")
print(f"Nombre de résultats trouvés : {len(results)}")
print("\nScores de similarité :")
print(results["similarities"].describe())


=== Exercise 3 : FAISS Indexing and Search ===

=== Création de l'index FAISS ===
Nombre de vecteurs dans l'index : 1000

=== Test de recherche ===
Recherche pour la requête 'animal':


,topic,link,domain,published_date,title,lang,extra_0,similarities
137,TECHNOLOGY,https://www.pushsquare.com/news/2020/08/random...,pushsquare.com,2020-08-03 16:30:00,Random: You Can Pick Up and Pet Cats in Assass...,en,None,0.391902
748,HEALTH,https://www.news-medical.net/news/20200813/Res...,news-medical.net,2020-08-13 05:18:00,Researchers explore social behavior of animals...,en,None,0.376784
80,TECHNOLOGY,https://www.gematsu.com/2020/08/ghostwire-toky...,gematsu.com,2020-08-07 16:43:13,Ghostwire: Tokyo confirms dog petting,en,None,0.344059
709,SCIENCE,https://www.thecut.com/2020/08/scientists-say-...,thecut.com,2020-08-04 12:52:00,Just Let This Lizard Be a Dinosaur,en,None,0.317387
585,SCIENCE,https://af.reuters.com/article/worldNews/idAFK...,af.reuters.com,2020-08-13 16:51:00,'Secret' life of sharks: Study reveals their s...,en,None,0.295497



=== Statistiques de recherche ===
Nombre de résultats trouvés : 5

Scores de similarité :
count    5.000000
mean     0.345126
std      0.040074
min      0.295497
25%      0.317387
50%      0.344059
75%      0.376784
max      0.391902
Name: similarities, dtype: float64


In [ ]:
import chromadb
from chromadb.config import Settings
import json
from IPython.display import display

print("=== Exercise 4 : ChromaDB Collection and Querying ===")

# 1. Initialisation avec configuration explicite
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    is_persistent=True
))

collection_name = "my_news"

try:
    # 2. Gestion de la collection
    if collection_name in [c.name for c in chroma_client.list_collections()]:
        print(f"Suppression de la collection existante : '{collection_name}'")
        chroma_client.delete_collection(name=collection_name)

    print(f"\nCréation de la collection : '{collection_name}'")
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"description": "Collection of news articles"}
    )

    # 3. Préparation des données
    raw_data = pdf_subset["topic;link;domain;published_date;title;lang"].str.split(';', expand=True)
    raw_data.columns = ['topic', 'link', 'domain', 'published_date', 'title', 'lang']

    documents = raw_data['title'][:100].tolist()
    topics = raw_data['topic'][:100].tolist()
    metadatas = [{"topic": topic, "source": "news_dataset"} for topic in topics]
    ids = [f"doc_{i}" for i in range(100)]

    # 4. Ajout des données avec feedback
    print(f"\nAjout de {len(documents)} documents...")
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

    # 5. Démonstration des capacités de recherche
    print("\nDémonstration des capacités de recherche :")

    # Recherche simple
    print("\na) Recherche simple pour 'space':")
    results = collection.query(
        query_texts=["space"],
        n_results=10
    )

    # Affichage structuré des résultats
    print("\nRésultats de la recherche :")
    for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
        print(f"\n{i+1}. Document:")
        print(f"   ID: {results['ids'][0][i]}")
        print(f"   Titre: {doc}")
        print(f"   Topic: {metadata['topic']}")
        if 'distances' in results:
            print(f"   Score de similarité: {1 - results['distances'][0][i]:.4f}")

    # 6. Démonstration de recherche avec filtres
    print("\nb) Recherche avec filtre par topic:")
    results_filtered = collection.query(
        query_texts=["technology"],
        n_results=5,
        where={"topic": "TECHNOLOGY"}
    )

    print("\nNombre total de documents dans la collection:", collection.count())

except Exception as e:
    print(f"Une erreur est survenue : {str(e)}")
    import traceback
    traceback.print_exc()


=== Exercise 4 : ChromaDB Collection and Querying ===

Création de la collection : 'my_news'
Une erreur est survenue : Length mismatch: Expected axis has 7 elements, new values have 6 elements


Traceback (most recent call last):
  File "<ipython-input-23-5ec7ad4e3268>", line 30, in <cell line: 0>
    raw_data.columns = ['topic', 'link', 'domain', 'published_date', 'title', 'lang']
    ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/generic.py", line 6313, in __setattr__
    return object.__setattr__(self, name, value)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "properties.pyx", line 69, in pandas._libs.properties.AxisProperty.__set__
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/generic.py", line 814, in _set_axis
    self._mgr.set_axis(axis, labels)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/internals/managers.py", line 238, in set_axis
    self._validate_set_axis(axis, new_labels)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/internals/base.py", line 98, in _validate_set_axis
    raise ValueError(
ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements


In [ ]:
# Import des bibliothèques nécessaires
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
print("=== Exercise 5 : Question Answering avec Hugging Face ===")

# Initialisation du modèle et du tokenizer
model_id = "gpt2-medium"  # Utilisation de gpt2-medium pour un bon équilibre performance/ressources
print(f"\nChargement du modèle {model_id}...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

# Création du pipeline
print("\nCréation du pipeline de génération...")
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

# Définition de la question et préparation du contexte
question = "What's the latest news on space development?"

# Comme nous avons eu une erreur avec ChromaDB, créons un contexte exemple
example_context = [
    "NASA announces new Mars mission for 2026",
    "SpaceX successfully launches 60 new Starlink satellites",
    "European Space Agency reveals plans for lunar base",
    "New telescope discovers potentially habitable exoplanet"
]
context = " ".join([f"#{str(i+1)}: {text}" for i, text in enumerate(example_context)])

# Création du prompt
prompt_template = f"""
Based on the following news articles, please answer the question.

Context:
{context}

Question: {question}

Answer:"""

print("\nGénération de la réponse...")
# Génération de la réponse
try:
    lm_response = pipe(prompt_template,
                      max_length=1024,
                      num_return_sequences=1,
                      temperature=0.7,
                      top_p=0.9)

    print("\nRéponse générée :")
    print(lm_response[0]["generated_text"])

except Exception as e:
    print(f"Une erreur est survenue lors de la génération : {str(e)}")

# Test avec une autre question
print("\nTest avec une autre question...")
second_question = "What are the major space companies working on?"
second_prompt = f"""
Based on the following news articles, please answer the question.

Context:
{context}

Question: {second_question}

Answer:"""

try:
    second_response = pipe(second_prompt,
                         max_length=1024,
                         num_return_sequences=1,
                         temperature=0.7,
                         top_p=0.9)

    print("\nRéponse à la seconde question :")
    print(second_response[0]["generated_text"])

except Exception as e:
    print(f"Une erreur est survenue lors de la génération : {str(e)}")


=== Exercise 5 : Question Answering avec Hugging Face ===

Chargement du modèle gpt2-medium...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Création du pipeline de génération...

Génération de la réponse...


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Réponse générée :

Based on the following news articles, please answer the question.

Context:
#1: NASA announces new Mars mission for 2026 #2: SpaceX successfully launches 60 new Starlink satellites #3: European Space Agency reveals plans for lunar base #4: New telescope discovers potentially habitable exoplanet

Question: What's the latest news on space development?

Answer: NASA's Mars mission will launch in 2026 and will return astronauts to the Red Planet in 2033.

#1: NASA announces new Mars mission for 2026

NASA's Mars mission will launch in 2026.

#2: SpaceX successfully launches 60 new Starlink satellites

SpaceX successfully launches 60 new Starlink satellites.

#3: European Space Agency reveals plans for lunar base

SpaceX successfully launches lunar base.

#4: New telescope discovers potentially habitable exoplanet

New telescope discovers potentially habitable exoplanet.

Question: What's the latest news on space development?

Answer: NASA's Mars mission will launch in 2